In [5]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import json
import gmaps
import os


# Import API key
#from api_keys import g_key


In [6]:
path = "output_data/cities.csv"
cities_df = pd.read_csv(path)
cities_df.head(3)

,City,Latitude,Longitude,Max Temp (K),Humidity,Sky Description,Cloudiness %,Wind speed,Country,Date,Max Temp (F)
0,Albany,42.60,-73.97,285.93,64,scattered clouds,39,1.57,US,1601605573,55.27
1,Te Anau,-45.42,167.72,283.58,77,overcast clouds,100,4.40,NZ,1601605573,51.04
2,Hobart,-42.88,147.33,293.15,34,scattered clouds,40,9.80,AU,1601605574,68.27


In [7]:
cities_df_new = cities_df[['Latitude','Longitude']]
cities_df_new.head(3)

,Latitude,Longitude
0,42.60,-73.97
1,-45.42,167.72
2,-42.88,147.33


In [123]:
locations = cities_df_new
humidity = cities_df['Humidity']
new_york_coordinates = (40.75, -74.00)
fig = gmaps.figure(center=new_york_coordinates, zoom_level=1.37)
fig.add_layer(gmaps.heatmap_layer(locations,weights=humidity))
fig

# var heatmap = new google.maps.visualization.HeatmapLayer({
#   data: heatmapData
# });
# heatmap.setMap(map);

Figure(layout=FigureLayout(height=&#39;420px&#39;))

In [17]:
!jupyter nbextension enable --py gmaps

Enabling notebook extension jupyter-gmaps/extension...
      - Validating: OK


In [9]:
cities_df.head(3)

,City,Latitude,Longitude,Max Temp (K),Humidity,Sky Description,Cloudiness %,Wind speed,Country,Date,Max Temp (F)
0,Albany,42.60,-73.97,285.93,64,scattered clouds,39,1.57,US,1601605573,55.27
1,Te Anau,-45.42,167.72,283.58,77,overcast clouds,100,4.40,NZ,1601605573,51.04
2,Hobart,-42.88,147.33,293.15,34,scattered clouds,40,9.80,AU,1601605574,68.27


In [51]:
cities_df.columns

Index([&#39;City&#39;, &#39;Latitude&#39;, &#39;Longitude&#39;, &#39;Max Temp (K)&#39;, &#39;Humidity&#39;,
       &#39;Sky Description&#39;, &#39;Cloudiness %&#39;, &#39;Wind speed&#39;, &#39;Country&#39;, &#39;Date&#39;,
       &#39;Max Temp (F)&#39;],
      dtype=&#39;object&#39;)

In [10]:
hotel_df = cities_df.loc[(cities_df["Max Temp (F)"] < 80) & (cities_df["Max Temp (F)"] > 70) \
    & (cities_df["Wind speed"]<10) & (cities_df["Cloudiness %"] == 0)].dropna()
hotel_df.head(3)


,City,Latitude,Longitude,Max Temp (K),Humidity,Sky Description,Cloudiness %,Wind speed,Country,Date,Max Temp (F)
24,Avarua,-21.21,-159.78,299.15,83,clear sky,0,7.20,CK,1601605581,79.07
53,Kulachi,31.93,70.46,299.52,14,clear sky,0,4.93,PK,1601605591,79.74
54,Rikitea,-23.12,-134.97,296.11,80,clear sky,0,8.30,PF,1601605591,73.60


In [12]:
latlng_1 = list(zip(hotel_df['Latitude'], hotel_df['Longitude']))
latlng_1  = [", ".join(map(str, x)) for x in latlng_1]

In [13]:
hotel_df['latlng'] = latlng_1
hotel_df_2 = hotel_df[['latlng', 'City','Country']]
hotel_df_2['Hotel Name'] = ""
hotel_df_2.reset_index()
hotel_df_2.head(3)

,latlng,City,Country,Hotel Name
24,"-21.21, -159.78",Avarua,CK,
53,"31.93, 70.46",Kulachi,PK,
54,"-23.12, -134.97",Rikitea,PF,


In [14]:
target_coords = []
latlngs = hotel_df_2['latlng']
for i in latlngs: 
    target_coords.append(f"{i}")

In [15]:
hotels = []
locations = []
lats = []
lngs = []


#what if I created a new colum of lat, long touples so I could iterate through the dataframe rather than a list? Would that make it easier 
params = {
    "radius": 5000,
    "type": 'lodging',
    "key": gkey
}

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # run a request using our params dictionary

for i in target_coords:

    response = requests.get(base_url, params=params).json()
    params["location"] = i
    # base url
    results = response['results']
    try: 
        # print(f"{results [0]['name']}")
        hotel = results [0]['name']
        hotels.append(hotel)
        # print(f"{results[0]['plus_code']['compound_code']}")
        location = results[0]['plus_code']['compound_code']
        locations.append(location)
        # print(f"{results[0]['geometry']['location']['lat']}")
        lat = results[0]['geometry']['location']['lat']
        lats.append(lat)
        # print(f"{results[0]['geometry']['location']['lng']}")
        lng = results[0]['geometry']['location']['lng']
        lngs.append(lng)
    except (KeyError, IndexError):
        pass


In [ ]:
#you're so close. try to do the same thing but in a dataframe. 
#create an empty dataframe to store api data. Start by creating empy strings



In [16]:
new_df = hotel_df_2[['latlng']]
new_df
new_df['Hotel Name'] = ""
new_df['Location'] = ""
new_df['Latitude'] = ""
new_df['Longitude'] = ""
new_df.head(3)
new_df = new_df.reset_index()
new_df.head(4)

,index,latlng,Hotel Name,Location,Latitude,Longitude
0,24,"-21.21, -159.78",,,,
1,53,"31.93, 70.46",,,,
2,54,"-23.12, -134.97",,,,
3,66,"29.11, 21.29",,,,


In [24]:
params = {
    "radius": 5000,
    "type": 'lodging',
    "key": gkey
}

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
for index, row in new_df.iterrows():
    target_coordinates = row['latlng']
    params['location'] = f"{target_coordinates}"
    print(f"Retrieving Results for Index {index}: {target_coordinates}.")
    response = requests.get(base_url, params=params).json()
    results = response['results']
    try: 
        new_df.loc[index, 'Hotel Name']  = results [0]['name']
        # print(f"{results[0]['plus_code']['compound_code']}")
        new_df.loc[index, 'Location'] = results[0]['plus_code']['compound_code']
        new_df.loc[index, 'Latitude'] = results[0]['geometry']['location']['lat']
        new_df.loc[index, 'Longitude'] = results[0]['geometry']['location']['lng']
        print(f"{results [0]['name']}")
    except (KeyError, IndexError):
        pass





Retrieving Results for Index 0: -21.21, -159.78.
Paradise Inn
Retrieving Results for Index 1: 31.93, 70.46.
SHAH JAHAN MAHSUD
Retrieving Results for Index 2: -23.12, -134.97.
Retrieving Results for Index 3: 29.11, 21.29.
Retrieving Results for Index 4: 12.99, 7.6.
Katsina Motel
Retrieving Results for Index 5: 31.72, -116.57.
Casa De Thompson
Retrieving Results for Index 6: -12.36, -44.97.
Pousada do Luizão
Retrieving Results for Index 7: -19.67, 22.27.
Retrieving Results for Index 8: -24.79, -50.01.
CHACARA BAILLY
Retrieving Results for Index 9: -38.33, 143.58.
Otway Gate Motel
Retrieving Results for Index 10: -37.83, 140.77.
Commodore on the Park
Retrieving Results for Index 11: 29.97, 40.21.
Fakhamat Al Orjoana Apartment
Retrieving Results for Index 12: 14.4, 46.47.
Retrieving Results for Index 13: -16.84, 36.99.
Phama Lodge
Retrieving Results for Index 14: -38.27, 141.97.
Codrington Gardens Bed And Breakfast
Retrieving Results for Index 15: -8.93, 125.41.
Juvinal Martin
Retrieving R

In [98]:
markers_df = new_df.drop(columns=['index'])
nan_value = float("NaN")
markers_df_2 = markers_df.replace("", nan_value, inplace=True)
c_markers_df = markers_df.dropna()
c_markers_df


,latlng,Hotel Name,Location,Latitude,Longitude
0,"-21.21, -159.78",Paradise Inn,"Q6VJ+V5 Avarua District, Cook Islands",-21.205333,-159.769621
1,"31.93, 70.46",SHAH JAHAN MAHSUD,"WFM6+4C Kulachi, Pakistan",31.932828,70.461043
4,"12.99, 7.6",Katsina Motel,"XHMX+8X Katsina, Nigeria",12.983333,7.600000
5,"31.72, -116.57",Casa De Thompson,"PC3R+9F Maneadero, Baja California, Mexico",31.703446,-116.558869
6,"-12.36, -44.97",Pousada do Luizão,"J2VC+9W São Desidério, State of Bahia, Brazil",-12.356610,-44.977709
8,"-24.79, -50.01",CHACARA BAILLY,"6297+P7 Jardim das Nacoes, Castro - State of P...",-24.780711,-49.986828
9,"-38.33, 143.58",Otway Gate Motel,"MH6X+R4 Colac VIC, Australia",-38.337924,143.597764
10,"-37.83, 140.77",Commodore on the Park,"5QGM+PJ Mount Gambier SA, Australia",-37.823155,140.784066
11,"29.97, 40.21",Fakhamat Al Orjoana Apartment,W5WV+H6 Sakaka Saudi Arabia,29.946421,40.193125
13,"-16.84, 36.99",Phama Lodge,"5X7P+W7 Mocuba, Mozambique",-16.835211,36.985726


In [101]:
c_markers_df['tuples'] = list(zip(c_markers_df.Latitude, c_markers_df.Longitude))
c_marker_locations = []
for x in c_markers_df['tuples']: 
   c_marker_locations.append(x)
c_marker_locations

[(-21.205333, -159.769621),
 (31.9328279, 70.4610427),
 (12.983333, 7.600000000000001),
 (31.70344589999999, -116.5588689),
 (-12.3566101, -44.9777094),
 (-24.7807113, -49.98682830000001),
 (-38.3379236, 143.5977641),
 (-37.823155, 140.784066),
 (29.9464214, 40.1931247),
 (-16.8352109, 36.98572579999999),
 (-38.2846732, 142.0097987),
 (-8.930576, 125.396797),
 (37.05580769999999, 140.8930947),
 (27.0380467, 14.4278714),
 (24.666677, 46.692159),
 (-25.5250061, -48.51193139999999),
 (31.2779214, 49.5990096),
 (8.407393699999998, 48.4978782),
 (-15.861252, 27.7490656),
 (29.03272369999999, 21.5458054),
 (-43.5265668, 172.6113713)]

In [121]:
marker_locations = [(-21.205333, -159.769621),
 (31.9328279, 70.4610427),
 (12.983333, 7.600000000000001),
 (31.70344589999999, -116.5588689),
 (-12.3566101, -44.9777094),
 (-24.7807113, -49.98682830000001),
 (-38.3379236, 143.5977641),
 (-37.823155, 140.784066),
 (29.9464214, 40.1931247),
 (-16.8352109, 36.98572579999999),
 (-38.2846732, 142.0097987),
 (-8.930576, 125.396797),
 (37.05580769999999, 140.8930947),
 (27.0380467, 14.4278714),
 (24.666677, 46.692159),
 (-25.5250061, -48.51193139999999),
 (31.2779214, 49.5990096),
 (8.407393699999998, 48.4978782),
 (-15.861252, 27.7490656),
 (29.03272369999999, 21.5458054),
 (-43.5265668, 172.6113713)] 

location = c_markers_df["Hotel Name"].tolist() 
hotel = c_markers_df["Location"].tolist() 
info_box_template = ""
fig = gmaps.figure()
for index, row in c_markers_df.iterrows():
    markers = gmaps.marker_layer(marker_locations,
    info_box_content=[f"Hotel: {row[1]} Location: {row[2]}" for index, row in c_markers_df.iterrows()]) 
    fig.add_layer(markers)
fig




Figure(layout=FigureLayout(height=&#39;420px&#39;))